In [ ]:
data_dir = ".\\data\\"

# SLC-2445, vac, 72hrs
filename = "2018-07-27-14-45-18.tdms"
keyence_filename = None

# SLC-1730, vac, 24hrs
#filename = "2018-07-26-14-39-24.tdms"
#keyence_filename = None

# SLC-1730, vac, 48hrs
#filename = "2018-07-30-16-19-44.tdms"
#keyence_filename = None

# SLC-2445, air, 30hrs, keyence
#filename = "2018-08-30-10-01-54.tdms"
#keyence_filename = "2018-08-30-10-03-14-KEYENCE.tdms"

# SLC-1730, air, 30hrs??, keyence
#filename = "2018-08-31-15-46-55.tdms"
#keyence_filename = "2018-08-31-15-46-59-KEYENCE.tdms"

#keyence_filename = None

#duty test SLC-1730
filename = "2018-09-27-09-10-21.tdms"
keyence_filename = None

#duty test SLC-2445
#filename = "2018-10-01-09-11-39.tdms"
#keyence_filename = None

In [ ]:
import batch_utils
import data_tools

In [ ]:
from ipywidgets import FloatProgress
from IPython.display import display

import numpy as np
from pprint import pprint
import datetime
from time import time

from importlib import reload
reload(batch_utils)
reload(data_tools)

full_data, info = batch_utils.load_data(filename, keyence_filename, suffix="full_data") 
pprint(info)
batch_utils.print_lengths(full_data, "Full dataset:")

In [ ]:
# This data is upside down!
if keyence_filename in set(["2018-08-31-15-46-59-KEYENCE.tdms", "2018-08-30-10-03-14-KEYENCE.tdms"]):
    full_data['keyence']['data1'] = full_data['keyence']['data1'] * -1

In [ ]:
# Fix jumps in time!
fix_jumps = set(["2018-07-27-14-45-18.tdms", "2018-07-26-14-39-24.tdms", "2018-08-31-15-46-55.tdms", "2018-09-27-09-10-21.tdms"])
if filename in fix_jumps:  
    breaks = ['set_point', 'temperature', 'position']
    
    if keyence_filename is not None:
        breaks.append('keyence')
        
    for key in breaks:
        print("\nLooking for the break in", key)
        time_diff = [(b-a) for a, b in zip(full_data[key]['time'], full_data[key]['time'][1:])]
        
        d_max  = np.max(time_diff)
        d_mean = np.mean(time_diff)
        d_min  = np.min(time_diff)

        print([
            ("Max ", d_max.total_seconds(), time_diff.index(d_max)), 
            ("Mean", d_mean.total_seconds()),
            ("Min ", d_min.total_seconds(), time_diff.index(d_min))
        ])
        
        if d_min.total_seconds() < 0: 
            broken = time_diff.index(d_min)

            print(key, "broken at", broken, time_diff[broken].total_seconds())

            diff = time_diff[broken] - time_diff[broken +1]

            fixed = []
            fixed.extend(full_data[key]['time'][:broken+1])
            fixed.extend([(v - diff) for v in full_data[key]['time'][broken+1:]])

            print("Fixed", key)

            time_diff = [(b-a) for a, b in zip(fixed, fixed[1:])]

            d_max  = np.max(time_diff)
            d_mean = np.mean(time_diff)
            d_min  = np.min(time_diff)

            print([
                ("Max ", d_max.total_seconds(), time_diff.index(d_max)), 
                ("Mean", d_mean.total_seconds()),
                ("Min ", d_min.total_seconds(), time_diff.index(d_min))
            ])

            full_data[key]['time'] = fixed
            
        else:
            print("Nothing to fix")

In [ ]:
limits = [full_data['set_point']['time'][48000], full_data['set_point']['time'][50000]]

data = dict(zip(full_data.keys(), [data_tools.truncate(limits, full_data[k], find_mode={'seek':False}) for k in full_data]))
batch_utils.print_lengths(data, "Truncated dataset:")

data=full_data

batch_utils.print_structure(data)

In [ ]:
# Initialise the index
indices = dict.fromkeys(data, 0)

# Start when all data streams have begun
current_time = max([data[d]['time'][0] for d in data])
print(current_time,)

def update_indices(new_time):
    change = set()
    for index in indices:
        try:
            while data[index]['time'][indices[index]+1] <= new_time:
                indices[index] += 1
                change.add(index)
        except IndexError:
            break
    return change

update_indices(current_time)

print(indices)        
pprint([str(data[index]['time'][indices[index]]) for index in indices])

# Sanity check all next times are greater than the current time
for index in indices:
    print(data[index]['time'][indices[index]+1] > current_time)

end = sum([len(data[d]['time']) for d in data])

print(end, "points to evaluate")

progress = FloatProgress(value=0, max=end/1000)
display(progress)

In [ ]:
# Initialise the stats...

data['set_point']['trans']        = [None,] * len(data['set_point']['time'])
data['set_point']['avg_position'] = [None,] * len(data['set_point']['time'])
data['set_point']['min_position'] = [None,] * len(data['set_point']['time'])
data['set_point']['max_position'] = [None,] * len(data['set_point']['time'])
data['set_point']['avg_keyence']  = [None,] * len(data['set_point']['time'])
data['set_point']['min_keyence']  = [None,] * len(data['set_point']['time'])
data['set_point']['max_keyence']  = [None,] * len(data['set_point']['time'])

#data['position']['round'] =         [None,] * len(data['position']['time'])
data['position']['set_point']     = [None,] * len(data['position']['time'])
data['position']['mask']          = [None,] * len(data['position']['time'])

if keyence_filename is not None:
    data['keyence']['set_point']  = [None,] * len(data['keyence']['time'])
    data['keyence']['mask']       = [None,] * len(data['keyence']['time'])

mask_next = datetime.timedelta(seconds = 0.5)
mask_prev = datetime.timedelta(seconds = 0.2)

# Position aggregation
pos_index_last_sp = indices['position']
pos_list = list()

if keyence_filename is not None:
    # Keyence aggregation
    key_index_last_sp = indices['keyence']
    key_list = list()

while sum([indices[index] for index in indices]) < end:
    
    # Position - setpoint
    data['position']['set_point'][indices['position']] = \
        data['position']['data'][indices['position']] \
        - data['set_point']['data'][indices['set_point']]

        
    # Keyence - setpoint
    if keyence_filename is not None:
        data['keyence']['set_point'][indices['keyence']] = \
            data['keyence']['data1'][indices['keyence']] \
            - data['set_point']['data'][indices['set_point']]
            
    # Transition time to within a micron   
    #if current_time == data['position']['time'][indices['position']] and \
    #                    data['set_point']['trans'][indices['set_point']] is None:
    #    
    #    if abs(data['position']['data'][indices['position']] - data['set_point']['data'][indices['set_point']]) < 0.001:
    #        data['set_point']['trans'][indices['set_point']] = data['position']['time'][indices['position']] \
    #                                    - data['set_point']['time'][indices['set_point']]
    
    # Transition time alternate method...
    #if current_time == data['set_point']['time'][indices['set_point']]:
    #    # Search for the next position which is close to the set point
    #    for i in range(indices['position']-1, indices['position'] + 1000):#len(data['position']['time']) - indices['position']):
    #        p = data['position']['data'][i]
    #        t = data['position']['time'][i]
    #        dif = abs(p - data['set_point']['data'][indices['set_point']])
    #        if dif < 0.001:
    #            data['set_point']['trans'][indices['set_point']] = t - data['set_point']['time'][indices['set_point']]
    #            break
                
    # Transition time, another method!
    if current_time == data['position']['time'][indices['position']] and \
                        data['set_point']['trans'][indices['set_point']] is None:
        if abs(data['position']['data'][indices['position']] - data['position']['data'][indices['position']+1]) < 0.001:
            data['set_point']['trans'][indices['set_point']] = current_time - data['set_point']['time'][indices['set_point']]

    # Aggregate reset
    if current_time == data['set_point']['time'][indices['set_point']]:
        # New set point value - save the values and reset!
        try:
            pos_list = pos_list[1:]
            data['set_point']['avg_position'][indices['set_point']-1] = np.mean(pos_list)
            data['set_point']['min_position'][indices['set_point']-1] = np.min(pos_list)
            data['set_point']['max_position'][indices['set_point']-1] = np.max(pos_list)
            
            data['position']['mask'][indices['position']-len(pos_list)-1] = True
            
            if keyence_filename is not None:
                key_list = key_list[4:]
                data['set_point']['avg_keyence'][indices['set_point']-1] = np.mean(key_list)
                data['set_point']['min_keyence'][indices['set_point']-1] = np.min(key_list)
                data['set_point']['max_keyence'][indices['set_point']-1] = np.max(key_list)
                
                data['keyence']['mask'][indices['keyence']-len(key_list)-1] = True
                data['keyence']['mask'][indices['keyence']-len(key_list)-2] = True
                data['keyence']['mask'][indices['keyence']-len(key_list)-3] = True
                data['keyence']['mask'][indices['keyence']-len(key_list)-4] = True
       
        except ValueError:
            print("Can't aggregate!")
            pprint(indices)
        
        pos_list = list()
        key_list = list()
    
    
    # Mask position
    try:
        data['position']['mask'][indices['position']] = (
            (data['position']['time'][indices['position']]) \
            < (data['set_point']['time'][indices['set_point']+1] - mask_next)) \
        and (
            (data['position']['time'][indices['position']]) \
            > (data['set_point']['time'][indices['set_point']] - mask_prev))
        
        pass
    except IndexError as e:
        # Run out of set_points
        next_set_time =    data['set_point']['time'][indices['set_point']] \
                        + (data['set_point']['time'][indices['set_point']] \
                         - data['set_point']['time'][indices['set_point']-1])
        
        data['position']['mask'][indices['position']] = (
            (data['position']['time'][indices['position']]) \
            < (next_set_time - mask_next)) \
        and (
            (data['position']['time'][indices['position']]) \
            > (data['set_point']['time'][indices['set_point']] - mask_prev))
        
        
    # Mask keyence
    if keyence_filename is not None:
        try:
            data['keyence']['mask'][indices['keyence']] = (
                (data['keyence']['time'][indices['keyence']]) \
                < (data['set_point']['time'][indices['set_point']+1] - mask_next)) \
            and (
                (data['keyence']['time'][indices['keyence']]) \
                > (data['set_point']['time'][indices['set_point']] - mask_prev))

            pass
        except IndexError as e:
            # Run out of set_points
            next_set_time =    data['set_point']['time'][indices['set_point']] \
                            + (data['set_point']['time'][indices['set_point']] \
                             - data['set_point']['time'][indices['set_point']-1])

            data['keyence']['mask'][indices['keyence']] = (
                (data['keyence']['time'][indices['keyence']]) \
                < (next_set_time - mask_next)) \
            and (
                (data['keyence']['time'][indices['keyence']]) \
                > (data['set_point']['time'][indices['set_point']] - mask_prev))
    
    
    
    
    next_time = min([data[index]['time'][min([indices[index]+1, len(data[index]['time'])-1])] for index in indices])
    
    old_indices = indices.copy()
    change = update_indices(next_time)
    
    # Aggregate updating
    if 'position' in change and not data['position']['mask'][old_indices['position']]:
        pos_list.append(data['position']['set_point'][old_indices['position']])
    
    if keyence_filename is not None:
        if 'keyence' in change and not data['keyence']['mask'][old_indices['keyence']]:
            key_list.append(data['keyence']['set_point'][old_indices['keyence']])
    
    
    #for index in indices:
    #    print(index, old_indices[index], indices[index], old_indices[index] == indices[index])
        
    if all([old_indices[index] == indices[index] for index in indices]):
        print("Stopping at {}".format(indices))
        break
    
    #print(sum([indices[index] for index in indices]), end)
    #print(current_time)
    #print(next_time)
    #print()
    
    current_time = next_time
    
    if sum([indices[index] for index in indices]) % 1000 == 0:
        progress.value += 1
    
    #break
    
progress.value = progress.max
print("Done!")

In [ ]:
print("Storing the processed data")
batch_utils.store_data(data, filename, keyence_filename, suffix="seq", info=info)
print("Done!")

data.pop('info')

In [ ]:
del full_data
del data